In [1]:
import openai
import re
import random
from collections import Counter

In [3]:
import json
import pandas as pd

json_file_path = 'data/MUStARD_data.json'

with open(json_file_path, 'r') as file:
    data = json.load(file)
df = pd.DataFrame.from_dict(data, orient='index')

utterances = df['utterance']
ans = df['sarcasm']
contexts = df['context']
speakers = df['context_speakers']
current_speaker = df['speaker']

In [4]:
outputs =[]

In [5]:
def getResultNoPrompt (speaker_dialogue, dialogue) :
    prompt = f"\"{' '.join(dialogue) + speaker_dialogue}\"\n You will be given movie or tv show dialogue. Determine whether the last statement is sarcastic with YES or NO."
    response = openai.chat.completions.create(
        model='gpt-4o-mini-2024-07-18',
        messages= [
            {"role": "user", "content" : prompt}
        ],
        max_tokens = 10,
        temperature = 1.0
    )
    output = response.choices[0].message.content.strip();
    isSarcastic = True if "yes" in output.lower() else False
    return isSarcastic;

In [6]:
def getResultWPrompt(speaker_dialogue, dialogue):
    judgeInput = ""  
    output = "" 
    
    #System prompt for sarcasm justification
    sarc_system_prompt = ("You will be given movie or tv show dialogue, and will analyze the statement marked between brackets. Summarize the conversation, and repeat back the statement to analyze."
                           "Then, analyze the following:\n"
                          "-What does the speaker imply about the situation with their statement?\n"
                          "-What does the speaker think about the situation?\n"
                          "-Are what the speaker implies and what the speaker thinks saying the same thing?\n"
                          "Finally, decide if the speaker is pretending to have a certain attitude toward the conversation."
                          )

    sarc_response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": sarc_system_prompt},
            {"role": "user", "content": ' '.join(dialogue) + "{"+ speaker_dialogue + "}"}
        ],
        max_tokens= 2000,
        temperature=1.0
    )
    print(' '.join(dialogue) + "{"+ speaker_dialogue + "}")
    judgeInput += "\n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n"
    judgeInput += sarc_response.choices[0].message.content.strip()

    reflect_system_prompt = ("You will be given a piece of movie dialogue, a statment marked in brackets, and a preliminary analysis on the marked statement. Summarize the preliminary analysis"
                             "Decide whether statement is sarcastic or not by first analyzing the following:\n"
                            "\nThe Implicature - What is implied in the conversation beyond the literal meaning?"
                            "\nThe Presuppositions - What information in the conversation is taken for granted?"
                            "\nThe intent of the speaker - What do the speaker(s) hope to achieve with their statement and who are the speakers?\n"
                            "\nThe polarity - Does the last sentence have a positive or negative tone?"
                            "\nPretense - Is there pretense in the speaker's attitude?"
                            "\nMeaning- What is the difference between the literal and implied meaning of the statement?"
                             "Reflect on the preliminary analysis and what should change, then decide if the statment is sarcastic.")
                            
                       
    reflection_response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": reflect_system_prompt},
            {"role": "user", "content": ' '.join(dialogue) + "{"+ speaker_dialogue + "}" + judgeInput}
        ],
        max_tokens=4000,
        temperature=1.0
    )
    
    judgeInput += "\n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n"
    judgeInput += reflection_response.choices[0].message.content.strip()
    judgeInput += "\n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n"
    output += (f"\nANSWER: {judgeInput}\n")

    clean_system_prompt = (
        "You will be given the output of an LLM which decided if a sentence is sarcastic or not. "
        "Read the output, then summarize the LLM's stance with ONLY a YES (they think the sentence is sarcastic) or NO (they think the sentence is not sarcastic)."
    )
    clean_output = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": clean_system_prompt},
            {"role": "user", "content": judgeInput}
        ],
        max_tokens=10,
        temperature=1.0
    )
    clean_output = clean_output.choices[0].message.content.strip()
    output += (f"Judgement: {clean_output}\n")

    # Store output and determine if the statement is sarcastic
    outputs.append(output)
    # Determine sarcasm based on clean output
    isSarcastic = True if "yes" in clean_output.lower() else False
    return isSarcastic




In [7]:
import random

def beginPrompting(inputs, answers,context, speakers, current_speaker, with_prompt, indices):
    num_samples = len(indices)
    with open("MUSTARD_reflection-mistakes.txt" ,"a") as file:
            file.write (f"\nIncorrect Answers: (Full output listed below) \n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n")

    sampled_inputs = [inputs.iloc[i] for i in indices]
    sampled_answers = [answers.iloc[i] for i in indices]
    sampled_contexts = [context.iloc[i] for i in indices]
    sampled_speakers = [speakers.iloc[i] for i in indices]
    sampled_current_speaker = [current_speaker.iloc[i] for i in indices]
    
    isSarCt = 0
    noSarCt = 0
    totNoSarCt = 0
    totSarCt = 0

    for x in range(num_samples):
        dialogue = [f"{speaker}: {utterance}" for speaker, utterance in zip(sampled_speakers[x], sampled_contexts[x])]
        speaker_dialogue = sampled_current_speaker[x] + ": " + sampled_inputs[x]
        isSarcastic = getResultWPrompt(speaker_dialogue, dialogue) if with_prompt else getResultNoPrompt(speaker_dialogue, dialogue)
        if sampled_answers[x]:
            totSarCt += 1
            if(isSarcastic):
                isSarCt += 1
            else:
                with open("MUSTARD_reflection-mistakes.txt" ,"a") as file:
                    file.write (f"\n{outputs[x]} \n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n")
                    file.write (f"\nANSWER: {sampled_answers[x]}\n")
        else:
            totNoSarCt += 1
            if(not isSarcastic):
                noSarCt += 1
            else:
                with open("MUSTARD_reflection-mistakes.txt" ,"a") as file:
                    file.write (f"\n{outputs[x]} \n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n")
                    file.write (f"\nANSWER: {sampled_answers[x]}\n")

        if with_prompt:
            outputs[x]+=(f"\nANSWER: {sampled_answers[x]}\n")
            outputs[x]+=(f"Prompt sarcastic statement recognition ({isSarCt} total)\n")
            outputs[x]+=(f"Total Prompted sarcastic statement  ({totSarCt} total)  \n")
            outputs[x]+=(f"Non sarcastic statement recognition ({noSarCt} total)\n")
            outputs[x]+=(f"Total Prompted non sarcastic statement ({totNoSarCt} total)\n")
            print("----------")
            print(f"Prompt sarcastic statement recognition ({isSarCt} total)")
            print(f"Total Prompted sarcastic statement  ({totSarCt} total)  ")
            print(f"Non sarcastic statement recognition ({noSarCt} total)")
            print(f"Total Prompted non sarcastic statement ({totNoSarCt} total)")
            print("----------")
             
    sar_percentage = (isSarCt / totSarCt * 100) if totSarCt > 0 else 0
    no_sar_percentage = (noSarCt / totNoSarCt * 100) if totNoSarCt > 0 else 0

    return sar_percentage, no_sar_percentage, totSarCt, totNoSarCt, noSarCt, isSarCt


In [8]:
sampleSize = 690
#total 690

In [9]:
random.seed(42)
indices = random.sample(range(len(utterances)), min(sampleSize, len(utterances)))
sar_percentage_prompted, no_sar_percentage_prompted, total_sar, total_no_sar, noSarCT, isSarCT = beginPrompting(utterances, ans, contexts, speakers, current_speaker, True, indices)
print("______________________________________________________________________")
print(f"Prompted sarcastic statement recognition: {sar_percentage_prompted}%")
print(f"Total correct: {isSarCT}")
print(f"Prompted non sarcastic statement recognition: {no_sar_percentage_prompted}%")
print(f"Total correct: {noSarCT}")
# sar_percentage_base, no_sar_percentage_base, total_sar_no_prompt, total_no_sar_no_prompt, zeroNoSarCT, zeroIsSarCT= beginPrompting(utterances, ans,contexts, speakers, current_speaker, False, indices)
# print(f"No prompt sarcastic statement recognition: {sar_percentage_base}%")
# print(f"Total correct: {zeroIsSarCT}")
# print(f"No prompt non sarcastic statement recognition: {no_sar_percentage_base}%")
# print(f"Total correct: {zeroNoSarCT}")


SCOTT: I finally got to admit it to myself. I have got a sarcasm problem.{MODERATOR: Well scott I am sure we are all really really looking forward to hear your story.}
----------
Prompt sarcastic statement recognition (1 total)
Total Prompted sarcastic statement  (1 total)  
Non sarcastic statement recognition (0 total)
Total Prompted non sarcastic statement (0 total)
----------
BERNADETTE: Are you guys doing okay? HOWARD: Leonard and I are fine, but I think Raj needs to meet a girl really soon. BERNADETTE: Well, that shouldn't be too hard. He's such a cutie. RAJ: Thank you, but "cute" is for bunnies. I want to be something with sex appeal, like... like a labradoodle.
Labradoodle?{LEONARD: We might be starting to zero in on your problem.}
----------
Prompt sarcastic statement recognition (1 total)
Total Prompted sarcastic statement  (2 total)  
Non sarcastic statement recognition (0 total)
Total Prompted non sarcastic statement (0 total)
----------
SHELDON: Pastries such as bear claws?

In [10]:
from datetime import datetime

now = datetime.now()

# Format the date and time as a string
formatted_datetime = now.strftime("%m-%d-%Y %H:%M:%S")

In [11]:
with open("MUSTARD_reflection.txt", "a") as file:
    file.write(f"\nMUStARD Dataset ({formatted_datetime}, Total Sample Size : {sampleSize}): \n")
    file.write(f"Prompted sarcastic statement recognition ({total_sar} total) : {sar_percentage_prompted}% \n")
    file.write(f"Total correct: {isSarCT}\n")
    file.write(f"Prompted non sarcastic statement recognition ({total_no_sar} total) : {no_sar_percentage_prompted}%\n")
    file.write(f"Total correct: {noSarCT}\n")
    # file.write(f"No prompt sarcastic statement recognition ({total_sar_no_prompt} total) : {sar_percentage_base}%\n")
    # file.write(f"Total correct: {zeroIsSarCT}")
    # file.write(f"No prompt non sarcastic statement recognition ({total_no_sar_no_prompt} total) : {no_sar_percentage_base}%\n")
    # file.write(f"Total correct: {zeroNoSarCT}")

In [12]:
with open("MUSTARD_reflection.txt" ,"a") as file:
    for x in range(len(outputs)) :
        file.write (f"\n{outputs[x]} \n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n")